In [1]:
import detectron2
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.data.datasets import register_coco_instances
import os
import json
import cv2
import random

In [ ]:


# Korak 1: Registracija COCO anotacija
data_path = 'path/to/your/data'
register_coco_instances("my_dataset_train", {}, f"/Users/kriskasira/Documents/RiProject/archive/Batch 3/import-videos/ann/instances.json", f"/Users/kriskasira/Documents/RiProject/archive/Batch 3/import-videos/images")

# Korak 2: Konfiguracija modela
cfg = get_cfg()
cfg.MODEL.DEVICE = "cpu"

cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 1000    # Prilagodite broj iteracija
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 7 #len(MetadataCatalog.get("my_dataset_train").thing_classes)  # Broj kategorija

# Korak 3: Treniranje modela
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

# Korak 4: Vizualizacija rezultata
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5   # Prilagodite prag
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
predictor = DefaultPredictor(cfg)

dataset_dicts = DatasetCatalog.get("my_dataset_train")
metadata = MetadataCatalog.get("my_dataset_train")

for d in random.sample(dataset_dicts, 3):    
    img = cv2.imread(d["file_name"])
    outputs = predictor(img)
    v = Visualizer(img[:, :, ::-1],
                   metadata=metadata, 
                   scale=0.8
    )
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2.imshow('Predictions', v.get_image()[:, :, ::-1])
    cv2.waitKey(0)
cv2.destroyAllWindows()


[07/16 15:51:29 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

[07/16 15:51:29 d2.data.datasets.coco]: Loaded 425 images in COCO format from /Users/kriskasira/Documents/RiProject/archive/Batch 3/import-videos/ann/instances.json
[07/16 15:51:29 d2.data.build]: Removed 0 images with no usable annotations. 425 images left.
[07/16 15:51:29 d2.data.build]: Distribution of instances among all 7 categories:
|   category    | #instances   |  category  | #instances   |  category  | #instances   |
|:-------------:|:-------------|:----------:|:-------------|:----------:|:-------------|
|    Player     | 3877         |    Ball    | 544          | Goal Line  | 325          |
|     Field     | 330          | Background | 428          |  Referee   | 969          |
| Football Pi.. | 655          |            |              |            |              |
|     total     | 7128         |            |              |            |              |
[07/16 15:51:29 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in training: [ResizeShortestEdge(short_edge_lengt

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (8, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (8,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (28, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (28,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (80, 256, 1, 1) in the checkpoint but (7, 256, 1, 1) 

[07/16 15:51:30 d2.engine.train_loop]: Starting training from iteration 0


/Users/kriskasira/anaconda3/lib/python3.11/site-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3550.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[07/16 15:54:31 d2.engine.hooks]: Overall training speed: 12 iterations in 0:02:33 (12.7787 s / it)
[07/16 15:54:31 d2.engine.hooks]: Total training time: 0:02:33 (0:00:00 on hooks)
[07/16 15:54:31 d2.utils.events]:  eta: 3:33:06  iter: 14  total_loss: 4.516  loss_cls: 2.052  loss_box_reg: 0.8218  loss_mask: 0.6949  loss_rpn_cls: 0.5801  loss_rpn_loc: 0.3748    time: 12.5939  last_time: 13.0031  data_time: 0.1828  last_data_time: 0.0036   lr: 3.4968e-06  
